In [10]:
from hdf5storage import loadmat, savemat 
import numpy as np 

from scipy import signal 
from scipy.fftpack import fft

from sklearn.covariance import GraphicalLassoCV

import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__)) 
# make sure it is newer version, works in version 1.2.1

The scikit-learn version is 1.2.1.


In [11]:
# load condition_all
outdict=loadmat('conditions_all.mat')
conditions_all=outdict["conditions_all"]

In [12]:
# Organize indicies 
Uncoupled_Ind=np.zeros((12,3))
L_Lead_Ind=np.zeros((12,3)) # they were leading indicies for L subject but following for R
R_Lead_Ind=np.zeros((12,3))
Mutual_Ind=np.zeros((12,3))

for ses in range(12):
    Uncoupled_Ind[ses]=np.asarray(np.where(conditions_all[ses]==1))
    L_Lead_Ind[ses]=np.asarray(np.where(conditions_all[ses]==2))
    R_Lead_Ind[ses]=np.asarray(np.where(conditions_all[ses]==3))
    Mutual_Ind[ses]=np.asarray(np.where(conditions_all[ses]==4))

In [13]:
# load abs_zscore_all
outdict=loadmat('abs_zscore_all.mat')
abs_zscore_all=outdict["abs_zscore_all"]

In [14]:
# append abs_zscore into 4 conditions in 2 syn types
# append all trials of the same condition together
synch_Uncoupled = list()
synch_Leading   = list()
synch_Following = list()
synch_Mutual    = list()

synco_Uncoupled = list()
synco_Leading   = list()
synco_Following = list()
synco_Mutual    = list()


for ses in list(range(0,12,2)): # synch [0, 2, 4, 6, 8, 10]
    for subj in range(2):
        for trl in range(3):
            if subj == 0:
                synch_Uncoupled.append(abs_zscore_all[ses][int(Uncoupled_Ind[ses][trl])][subj])
                synch_Leading.append(abs_zscore_all[ses][int(L_Lead_Ind[ses][trl])][subj])  
                synch_Following.append(abs_zscore_all[ses][int(R_Lead_Ind[ses][trl])][subj])    
                synch_Mutual .append(abs_zscore_all[ses][int(Mutual_Ind[ses][trl])][subj])  
            else:
                synch_Uncoupled.append(abs_zscore_all[ses][int(Uncoupled_Ind[ses][trl])][subj])
                synch_Leading.append(abs_zscore_all[ses][int(R_Lead_Ind[ses][trl])][subj])  
                synch_Following.append(abs_zscore_all[ses][int(L_Lead_Ind[ses][trl])][subj])   
                synch_Mutual .append(abs_zscore_all[ses][int(Mutual_Ind[ses][trl])][subj])  

for ses in list(range(1,12,2)): # [1, 3, 5, 7, 9, 11]
    for subj in range(2):
        for trl in range(3):
            if subj == 0:
                synco_Uncoupled.append(abs_zscore_all[ses][int(Uncoupled_Ind[ses][trl])][subj])
                synco_Leading.append(abs_zscore_all[ses][int(L_Lead_Ind[ses][trl])][subj])  
                synco_Following.append(abs_zscore_all[ses][int(R_Lead_Ind[ses][trl])][subj])    
                synco_Mutual .append(abs_zscore_all[ses][int(Mutual_Ind[ses][trl])][subj])  
            else:
                synco_Uncoupled.append(abs_zscore_all[ses][int(Uncoupled_Ind[ses][trl])][subj])
                synco_Leading.append(abs_zscore_all[ses][int(R_Lead_Ind[ses][trl])][subj])  
                synco_Following.append(abs_zscore_all[ses][int(L_Lead_Ind[ses][trl])][subj])
                synco_Mutual .append(abs_zscore_all[ses][int(Mutual_Ind[ses][trl])][subj])


#  combine the 4 conditions each concatenated in a list
synch_4condi=[np.concatenate(synch_Uncoupled), np.concatenate(synch_Leading), np.concatenate(synch_Following), np.concatenate(synch_Mutual)]
synco_4condi=[np.concatenate(synco_Uncoupled), np.concatenate(synco_Leading), np.concatenate(synco_Following), np.concatenate(synco_Mutual)]
syn_4condi=[synch_4condi, synco_4condi]


In [129]:
def partialcorrelation(ampdata,ncv=5):
    pc_lasso = GraphicalLassoCV(cv=ncv)
    nbin = np.shape(ampdata)[1]
    nchan = np.shape(ampdata)[2]
    pcorr = np.zeros((nbin,nchan,nchan))
    alphas = np.zeros(nbin)
    # likelihoods = list()
    likelihoods = np.zeros(nbin)
    covmat = np.zeros((nbin,nchan,nchan))
    for f in range(nbin):
        xx= np.abs(np.squeeze(ampdata[:,f,:]))
        pc_lasso.fit(xx) # the system is too ill for this solver
        covariance = pc_lasso.covariance_
        precision = pc_lasso.precision_
        alphas[f] = pc_lasso.alpha_
        dict_cvresults=pc_lasso.cv_results_
        best_alpha_ind=np.where(dict_cvresults['alphas']==pc_lasso.alpha_)
        # likelihoods.append(pc_lasso.cv_results_["mean_test_score"])
        likelihoods[f] = pc_lasso.cv_results_["mean_test_score"][best_alpha_ind][0]
        parcor = np.zeros((32,32))
        y = np.diag(precision)
        yy = np.outer(y,y)
        yy = np.sqrt(yy)
        parcor = precision/yy
        pcorr[f,:,:] = parcor
        y = np.diag(covariance)
        yy = np.outer(y,y)
        yy = np.sqrt(yy)
        parcor_cov = covariance/yy        
        covmat[f,:,:] = parcor_cov 
    return pcorr,covmat,alphas,likelihoods  


In [19]:
# parameters 
epoch = 1 #units: seconds
maxf = 30 #units: Hz
maxbin = maxf*epoch #convert maxf into number of bins to save

In [130]:
# fit lasso model for each of the 4 conditions in 2 syn types
synch_pcorr =   np.zeros((4,maxbin,32,32));       synco_pcorr  =np.zeros((4,maxbin,32,32))
synch_covmat=   np.zeros((4,maxbin,32,32));       synco_covmat =np.zeros((4,maxbin,32,32))
synch_alpha =   np.zeros((4,maxbin));             synco_alpha  =np.zeros((4,maxbin))
synch_likelihood = np.zeros((4,maxbin)) #list()      
synco_likelihood = np.zeros((4,maxbin)) #list()
for condi in range(4):
    pcorr,covmat,alphas,likelihoods = partialcorrelation(synch_4condi[condi],ncv=5)
    synch_pcorr[condi,:,:,:]  =pcorr
    synch_covmat[condi,:,:,:] =covmat
    synch_alpha[condi,:]    =alphas
    # synch_likelihood.append(likelihoods)
    synch_likelihood[condi,:] =likelihoods
    pcorr,covmat,alphas,likelihoods = partialcorrelation(synco_4condi[condi],ncv=5)
    synco_pcorr [condi,:,:,:] =pcorr
    synco_covmat[condi,:,:,:] =covmat
    synco_alpha[condi,:]    =alphas
    # synco_likelihood.append(likelihoods)
    synco_likelihood[condi,:] =likelihoods
outdict = dict()
outdict['synch_pcorr'] = synch_pcorr
outdict['synco_pcorr'] = synco_pcorr
outdict['synch_covmat']= synch_covmat, 
outdict['synco_covmat'] = synco_covmat
outdict['synch_alpha'] = synch_alpha
outdict['synco_alpha'] = synco_alpha
outdict['synch_likelihood'] = synch_likelihood
outdict['synco_likelihood'] = synco_likelihood

# takes 3 min

/home/zhibinz2/anaconda3/lib/python3.9/site-packages/sklearn/covariance/_graph_lasso.py:272: RuntimeWarning: invalid value encountered in multiply
  precision_[indices != idx, idx] = -precision_[idx, idx] * coefs
/home/zhibinz2/anaconda3/lib/python3.9/site-packages/sklearn/covariance/_graph_lasso.py:273: RuntimeWarning: invalid value encountered in multiply
  precision_[idx, indices != idx] = -precision_[idx, idx] * coefs
/home/zhibinz2/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:230: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/home/zhibinz2/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:230: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/home/zhibinz2/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:230: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/home/zhibinz2/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.

In [ ]:
# save syn_pcorr

savemat('syn_pcorr',outdict,store_python_metadata=True)
